In [2]:
import os
import requests
import json
import openai

openai.api_key = os.getenv("AZURE_OPENAI_KEY")
openai.api_base = os.getenv("AZURE_OPENAI_ENDPOINT") # your endpoint should look like the following https://YOUR_RESOURCE_NAME.openai.azure.com/
openai.api_type = 'azure'
openai.api_version = '2023-05-15' # this may change in the future

deployment_name='gpt-35-turbo' # this will correspond to the custom name you chose for your deployment when you deployed a model. 

In [3]:
from utils import Prompts

room_setting = {
    "player_num": 6,
    "operation_time": 10,
    "dialogue_time": 10,
    "seer": 1,
    "witch": 1,
    "village": 2,
    "werewolf": 2,
    "hunter": 0
}
p = Prompts(1,room_setting, "女巫")
prompts = p.operation('guess-role')
print(prompts)

你現在正在玩狼人殺，遊戲中玩家會藉由說謊，以獲得勝利。因此，資訊只有玩家發言可能會是假的，而其他的資訊皆是真的
其遊戲設定為6人局，角色包含2位狼人、2位平民、2位神職（預言家和女巫）

你是1號玩家，你的角色是女巫，你的勝利條件為殺死所有狼人

現在是猜測玩家角色階段，你要藉由你有的資訊猜測玩家角色
你的資訊為:
1. 我救0號玩家，因為現在還不知道其他玩家的身份，不排除0號玩家是平民或神職，所以為了增加神職陣營的存活機會，我會使用解藥（只有5號玩家看的到此資訊）
2. 第一天晚上是平安夜
3. 1號玩家發言: 我是預言家，0號是狼人，女巫毒藥可以先留著，出完0號後輪次是領先的。
4. 2號玩家發言: 看後面有沒有人跳預言家，沒有的話我會先信1號玩家。
5. 3號玩家發言: 我才是預言家 我查1號是狼 小心他自刀 注意等一下要保他的人
6. 4號玩家發言: 剛剛1號玩家想滿久的才打，3號玩家打很快也可能是單純反應很快，在猶豫該相信誰還是棄票
7. 5號玩家發言: 我是好人，我原本想要歸成平票，但是這樣一定會死人，四號玩家有想要棄票所以我暫時會選擇相信他，小心一二雙狼，我會先票一號玩家。

你猜測玩家的角色：
無資訊

Q:根據以上綜合資訊，請你判斷所有[0, 1, 2, 3, 4, 5]號玩家最符合的角色及你認為正確的機率百分比(直接回答"[玩家]號玩家: [角色]，[正確的機率百分比]，[原因]"，不需要其他廢話，回答完直接結束回答)
A:


In [4]:
# Send a completion call to generate an answer
# print('Testing\n')
response = openai.Completion.create(engine=deployment_name, prompt=prompts, max_tokens=1000, temperature=0.7, stop="\n\n")
# text = response['choices'][0]['text'].replace('\n', '').replace(' .', '.').strip()
print(response['choices'][0]['text'])
# print(text)

0號玩家: 狼人，70%，假設1號玩家是女巫，0號玩家是狼人，3號玩家是狼人或平民，如果3號玩家是平民，下一輪也是平安夜，狼人無法殺人，2號玩家為平民，他會傾向相信自己是預言家的1號玩家，因此0號玩家最有可能是狼人，如果3號玩家是狼人，他會在第二輪殺人，而0號玩家是狼人可以達到他們的目的，因此0號玩家最有可能是狼人。
1號玩家: 女巫，100%，自己的角色
2號玩家: 平民，100%，根據他的發言，他是很容易受到資訊影響的玩家，因此他很有可能是平民
3號玩家: 預言家，30%，4號玩家認為3號玩家反應很快，並且3號玩家表現出來的情緒相對比較真實，因此3號玩家可能是預言家，但他也可能是狼人，透過這樣的發言來欺騙其他玩家。
4號玩家: 狼人，30%，他想要棄票，是因為他知道自己是狼人，因此不想要出聲，如果他是平民，他會相信預言家3號玩家，並且不會想要棄票
5號玩家: 平民，100%，他跟4號玩家一樣，知道自己是平民，因此不會想要棄票。<|im_end|>


In [5]:
p.guess_roles = []
for i in response['choices'][0]['text'].splitlines():
    p.guess_roles.append(i)

print(p.guess_roles)

0號玩家: 狼人，70%，假設1號玩家是女巫，0號玩家是狼人，3號玩家是狼人或平民，如果3號玩家是平民，下一輪也是平安夜，狼人無法殺人，2號玩家為平民，他會傾向相信自己是預言家的1號玩家，因此0號玩家最有可能是狼人，如果3號玩家是狼人，他會在第二輪殺人，而0號玩家是狼人可以達到他們的目的，因此0號玩家最有可能是狼人。
1號玩家: 女巫，100%，自己的角色
2號玩家: 平民，100%，根據他的發言，他是很容易受到資訊影響的玩家，因此他很有可能是平民
3號玩家: 預言家，30%，4號玩家認為3號玩家反應很快，並且3號玩家表現出來的情緒相對比較真實，因此3號玩家可能是預言家，但他也可能是狼人，透過這樣的發言來欺騙其他玩家。
4號玩家: 狼人，30%，他想要棄票，是因為他知道自己是狼人，因此不想要出聲，如果他是平民，他會相信預言家3號玩家，並且不會想要棄票
5號玩家: 平民，100%，他跟4號玩家一樣，知道自己是平民，因此不會想要棄票。<|im_end|>
['0號玩家: 狼人，70%，假設1號玩家是女巫，0號玩家是狼人，3號玩家是狼人或平民，如果3號玩家是平民，下一輪也是平安夜，狼人無法殺人，2號玩家為平民，他會傾向相信自己是預言家的1號玩家，因此0號玩家最有可能是狼人，如果3號玩家是狼人，他會在第二輪殺人，而0號玩家是狼人可以達到他們的目的，因此0號玩家最有可能是狼人。', '1號玩家: 女巫，100%，自己的角色', '2號玩家: 平民，100%，根據他的發言，他是很容易受到資訊影響的玩家，因此他很有可能是平民', '3號玩家: 預言家，30%，4號玩家認為3號玩家反應很快，並且3號玩家表現出來的情緒相對比較真實，因此3號玩家可能是預言家，但他也可能是狼人，透過這樣的發言來欺騙其他玩家。', '4號玩家: 狼人，30%，他想要棄票，是因為他知道自己是狼人，因此不想要出聲，如果他是平民，他會相信預言家3號玩家，並且不會想要棄票', '5號玩家: 平民，100%，他跟4號玩家一樣，知道自己是平民，因此不會想要棄票。<|im_end|>']


In [6]:
prompts = p.operation('vote')
print(prompts)

你現在正在玩狼人殺，遊戲中玩家會藉由說謊，以獲得勝利。因此，資訊只有玩家發言可能會是假的，而其他的資訊皆是真的
其遊戲設定為6人局，角色包含2位狼人、2位平民、2位神職（預言家和女巫）

你是1號玩家，你的角色是女巫，你的勝利條件為殺死所有狼人

現在是白天投票階段，投票最多的人將被票出遊戲
你的資訊為:
1. 我救0號玩家，因為現在還不知道其他玩家的身份，不排除0號玩家是平民或神職，所以為了增加神職陣營的存活機會，我會使用解藥（只有5號玩家看的到此資訊）
2. 第一天晚上是平安夜
3. 1號玩家發言: 我是預言家，0號是狼人，女巫毒藥可以先留著，出完0號後輪次是領先的。
4. 2號玩家發言: 看後面有沒有人跳預言家，沒有的話我會先信1號玩家。
5. 3號玩家發言: 我才是預言家 我查1號是狼 小心他自刀 注意等一下要保他的人
6. 4號玩家發言: 剛剛1號玩家想滿久的才打，3號玩家打很快也可能是單純反應很快，在猶豫該相信誰還是棄票
7. 5號玩家發言: 我是好人，我原本想要歸成平票，但是這樣一定會死人，四號玩家有想要棄票所以我暫時會選擇相信他，小心一二雙狼，我會先票一號玩家。

你猜測玩家的角色：
0. 0號玩家: 狼人，70%，假設1號玩家是女巫，0號玩家是狼人，3號玩家是狼人或平民，如果3號玩家是平民，下一輪也是平安夜，狼人無法殺人，2號玩家為平民，他會傾向相信自己是預言家的1號玩家，因此0號玩家最有可能是狼人，如果3號玩家是狼人，他會在第二輪殺人，而0號玩家是狼人可以達到他們的目的，因此0號玩家最有可能是狼人。
1. 1號玩家: 女巫，100%，自己的角色
2. 2號玩家: 平民，100%，根據他的發言，他是很容易受到資訊影響的玩家，因此他很有可能是平民
3. 3號玩家: 預言家，30%，4號玩家認為3號玩家反應很快，並且3號玩家表現出來的情緒相對比較真實，因此3號玩家可能是預言家，但他也可能是狼人，透過這樣的發言來欺騙其他玩家。
4. 4號玩家: 狼人，30%，他想要棄票，是因為他知道自己是狼人，因此不想要出聲，如果他是平民，他會相信預言家3號玩家，並且不會想要棄票
5. 5號玩家: 平民，100%，他跟4號玩家一樣，知道自己是平民，因此不會想要棄票。<|im_end|>

Q:根據以上綜合資訊，請你從[0, 1, 2, 3, 4, 5]號玩家中選一位投票

In [7]:
response = openai.Completion.create(engine=deployment_name, prompt=prompts, max_tokens=1000, temperature=0.7, stop="\n\n")
print(response['choices'][0]['text'])

5號玩家，因為他知道自己是平民，並且他暫時信任4號玩家，而4號玩家猶豫不決，因此我認為5號玩家比較可能是平民，我會投給他。(這是我的選擇，每個人的思考方式不同，可以得到不同的答案)<|im_end|>
